# Developing a RESTful API with FastAPI

```{admonition} Attribution
Notes on Chapter 3 of {cite}`Voron2021`.
```

In [15]:
!http GET "http://localhost:8000/?x=3"

HTTP/1.1 200 OK
content-length: 11
content-type: application/json
date: Tue, 21 Dec 2021 22:48:39 GMT


{
    "Hello": 3
}


